# Portfolio Project: Safety Option

**Names:** Deniz Makul, Richard Sha, Mitchel Shen

**Due:** 11/25/2022 at 11:59pm ET on Dropbox

**What to submit:** your .ipynb (python code in jupyter notebooks) file as well as a pdf with all the output from your code.  




In [3]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

In [8]:
#Getting tickers from csv file
csv_tickers = pd.read_csv('Tickers_Example.csv', header = None)

In [9]:
# Richard
csv_tickers

,0
0,AAPL
1,ABBV
2,ABT
3,ACN
4,AGN
...,...
57,TXN
58,UNH
59,UNP
60,UPS


In [ ]:
# Deniz

In [ ]:
# Mitchel